In [12]:
# libraries
import plotly
import warnings
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from PIL import ImageColor

warnings.simplefilter(action='ignore', category=FutureWarning) # ignores warnings from pd.append()

In [285]:
# import csv
df = pd.read_csv('dff.csv')
df.tail()

,Titel,Bevilliget beløb,Modtager,Institution,Virkemidler,Område,År,Beskrivelse,Region
4426,Enabling Ultra Deep Hydrodesulphurization by N...,10781874,Ib Chorkendorff,Danmarks Tekniske Universitet,Øvrige forskningsprojekter,Teknologi og Produktion,2013,Alle olieprodukter renses i dag for svovl for ...,Region Hovedstaden
4427,Acute stroke research,717359,Hanne Krarup Christensen,"Bispebjerg Hospital, Neurologisk Afdeling",Delestillinger,Sundhed og Sygdom,2013,Aktuelle ansøgning angår frikøb af overlæge Ha...,Region Hovedstaden
4428,Atherosclerotic cardiovascular disease in HIV-...,764683,Anne-Mette Lebech,"Hvidovre Hospital, Infektionsmedicinsk Afdeling",Delestillinger,Sundhed og Sygdom,2013,Behandling af HIV positive patienter med anti-...,Region Hovedstaden
4429,Epigenetic modulation of mechanisms involved i...,829294,Ole Schmeltz Søgaard,Aarhus Universitetshospital,Delestillinger,Sundhed og Sygdom,2013,HIV infektion behandles i dag med en kombinati...,Region Midtjylland
4430,Novel mechanisms of insulin resistance and mit...,665923,Kurt Højlund,"Odense Universitetshospital, Endokrinologisk A...",Delestillinger,Sundhed og Sygdom,2013,Insulinresistens (IR) i muskelvæv spiller en v...,Region Syddanmark


In [336]:
# function to generate sankey diagram 
def generateSankey(df, category_columns=None, is_year: bool = True, year=None, is_comparisson: bool = False, comparer_institution=None):

    colorpalette = px.colors.qualitative.Dark24 + px.colors.qualitative.Light24 + px.colors.qualitative.Alphabet # hex values
    opacity = 0.5

    # data for sankey
    if is_year == False:
        df = df.loc[df['År'] == year] # get data for specified year argument

    if is_comparisson == True:
        category_columns.append('Institution') # add instituion to category_columns argument
        df_sankey = df.loc[:,category_columns + ['Bevilliget beløb']] # if True we include the Institution column to compare
        df_sankey = df_sankey[df_sankey['Institution'].isin(comparer_institution)] # after selecting the Institution column, we sort for the two institutions we want to compare
    else:
        df_sankey = df.loc[:,category_columns + ['Bevilliget beløb']] # if False we do not include the Institution column

    # create list of labels, i.e. unique values from each column except the values
    labels = []

    for col in category_columns:
        labels = labels + list(set(df_sankey[col].values)) # adds unique labels in each category to list

    # define colors based on number of labels
    color_dict_labels = dict(zip(labels, colorpalette)) # zips labels list with colorpalette

    # initiate input for for loop
    df_link_input = pd.DataFrame({'source' : [], 'target': [], 'count': []})

    # create data for go.Sankey function
    for i in range(len(category_columns)-1):
        if len(category_columns) == 1:
            print("Number of input categories must be at least 2")
        else:
            temporary_df = df_sankey.groupby([category_columns[i], category_columns[i+1]]).agg({'Bevilliget beløb':'sum'}).reset_index() # loop over columns and group by column to the right, i.e. 'År' and 'Virkemidler', and then 'Virkemidler' and 'Område'
            temporary_df.columns = ['source','target','count']
            df_link_input = df_link_input.append(temporary_df)

    # add index for source-target pair
    df_link_input['sourceID'] = df_link_input['source'].apply(lambda x: labels.index(x))
    df_link_input['targetID'] = df_link_input['target'].apply(lambda x: labels.index(x))

    # define colors based on source
    colorlist_source = [color_dict_labels[i] for i in df_link_input['source']] # loops over source column, and finds the value in the dictionary, and appends it to a new list. For example, the first 8 sources are 2022, so we append the value paired with our key, i.e. 2022.
    colorlist_source_rgba = ["rgba" + str(ImageColor.getcolor(color, "RGB") + (opacity, )) for color in colorlist_source] # converts hex colors to rgba

    # creating the sankey diagram
    fig = go.Figure(data=[go.Sankey(
        valueformat = ",",
        valuesuffix = " kr.",
        # define nodes
        node = dict(
            pad = 15,
            thickness = 20,
            line = dict(color = "black", width = 0.5),
            label = labels,
            color = list(color_dict_labels.values())
            ),
        link = dict(
            source = df_link_input['sourceID'], # indices correspond to labels, e.g. '2022', 'Forskningsprojekt 1', 'Forskningsprojekt 2', ...
            target = df_link_input['targetID'],
            value = df_link_input['count'],
            color = colorlist_source_rgba
        ))])

    fig.update_layout(title_text="Funding of Research Grants in " + str(year), font_size=10)
    return fig

In [342]:
# plotting sankey diagram
generateSankey(df, is_year = False, year = 2022, category_columns = ['År','Virkemidler', 'Område'], is_comparisson=True, comparer_institution=['Københavns Universitet', 'Syddansk Universitet', 'Aalborg Universitet'])

In [351]:

# stacked area chart
def generateStacked_categories(df, institution_list):
    
    # categories Område - only the six biggest
    categories_list = ['Kultur og Kommunikation', 'Natur og Univers',
                       'Samfund og Erhverv', 'Sundhed og Sygdom',
                       'Teknologi og Produktion', 'Tværrådslig']

    # data for stacked
    df_stacked = df.groupby(['År', 'Område', 'Institution']).agg({'Bevilliget beløb':'sum'}).reset_index()
    df_stacked_category = df_stacked[df_stacked['Område'].isin(categories_list)]

    df_stacked_all = df_stacked_category.groupby(['År', 'Område']).agg({'Bevilliget beløb':'sum'}).reset_index() # data for All Periods
    df_stacked_instituion = df_stacked_category[df_stacked_category['Institution'].isin(institution_list)]

    # Comparisson variables
    if len(institution_list)==2:
        titel = 'Funding over time for ' + str(institution_list[0]) + ' and ' + str(institution_list[1])
        height, width = 600, 1000
    else:
        titel = 'Funding over time for ' + str(institution_list[0])
        height, width = 400, 1000

    # figure
    if any("All Periods" in string for string in institution_list): # if All Periods is checked of new figure
        fig = px.area(df_stacked_all,
                  x="År",
                  y='Bevilliget beløb',
                  facet_col='Område',
                  color='Område',
                  title=titel,
                  height=height, width=width)
    else:
        fig = px.area(df_stacked_instituion,
                  x="År",
                  y='Bevilliget beløb',
                  facet_col='Område',
                  color='Område',
                  facet_row='Institution',
                  title=titel,
                  height=height, width=width)   

    # matches axes for every row
    fig.layout.yaxis.matches = 'y'
    fig.layout.yaxis2.matches = 'y'
    fig.layout.yaxis3.matches = 'y'
    fig.layout.yaxis4.matches = 'y'
    fig.layout.yaxis5.matches = 'y'
    fig.layout.yaxis6.matches = 'y'
    
    if len(institution_list)==2:
        fig.layout.yaxis7.matches = 'y7'
        fig.layout.yaxis8.matches = 'y7'
        fig.layout.yaxis9.matches = 'y7'
        fig.layout.yaxis10.matches = 'y7'
        fig.layout.yaxis11.matches = 'y7'
        fig.layout.yaxis12.matches = 'y7'

    # remove '=' from titles
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

    # font size
    fig.update_layout(font = dict(size = 10, color = "Black"))

    # hide legend
    fig.update_layout(showlegend=False)

    return fig

generateStacked_categories(df, institution_list=['Aalborg Universitet', 'Syddansk Universitet'])

# funding total skal kunne vælge for alle


In [349]:
def generateStacked_total(df, institution_list):
    
    # data
    df_stacked = df.groupby(['År', 'Institution']).agg({'Bevilliget beløb':'sum'}).reset_index()
    df_stacked_institution = df_stacked[df_stacked['Institution'].isin(institution_list)]
    df_stacked_all = df.groupby(['År']).agg({'Bevilliget beløb':'sum'}).reset_index() # data for All Periods

    # Comparisson variables
    if len(institution_list)==2:
        titel = 'Funding over time for ' + str(institution_list[0]) + ' and ' + str(institution_list[1])
        height, width = 600, 1000
    else:
        titel = 'Funding over time for ' + str(institution_list[0])
        height, width = 500, 1000

    # figure
    if any("All Periods" in string for string in institution_list): # if All Periods is checked of new figure
        fig = px.area(df_stacked_all,
                  x="År",
                  y='Bevilliget beløb',
                  title=titel,
                  height=500, width=700)
    else:
        fig = px.area(df_stacked_institution,
                    x='År',
                    y='Bevilliget beløb',
                    title=titel,
                    height=height, width=width,
                    facet_col='Institution',
                    color='Institution')
    
    # remove '=' from titles
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

    # hide legend
    fig.update_layout(showlegend=False)
    
    return fig

generateStacked_total(df, institution_list=['Københavns Universitet'])